In [1]:
from __future__ import print_function
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import random
import os
import matplotlib.pyplot as plt

In [60]:
class create_input:
    def Cutout(self, img):
        # choose one or three rectangle with random size(up to 1/5 image size)
        print(tf.shape(img))
        height = tf.shape(img)[0]
        width = tf.shape(img)[1]
        h = tf.random.uniform([], int(height/10), int(height/5), tf.int32)
        w = tf.random.uniform([], int(width/10), int(width/5), tf.int32)

        x1 = tf.random.uniform([], 0, height+1 - h, tf.int32)
        y1 = tf.random.uniform([], 0, width+1 - w, tf.int32)

        mean = int(tf.reduce_mean(img))
        erase_area = tf.cast(tf.fill([h, w, 3], 230), tf.uint8)
        erase_img = np.array(img)
        erase_img[x1:x1+h, y1:y1+w, :] = erase_area

        return erase_img

    def WhiteNoise(self, img):
        noise = tf.random.normal(tf.shape(img), mean=0.0, stddev=20.0, dtype=tf.float32)
        noise_img = noise+img
        noise_img = tf.clip_by_value(noise_img, 0.0, 255.0)

        return noise_img

    def SaltNoise(self, img):
        # add salt and pepper noise(1/16 area)
        pts_x_white = np.random.randint(0, tf.shape(img)[0]-1, int(tf.shape(img)[0]**2/16))
        pts_y_white = np.random.randint(0, tf.shape(img)[1]-1, int(tf.shape(img)[0]**2/16))

        pts_x_black = np.random.randint(0, tf.shape(img)[0]-1, int(tf.shape(img)[0]**2/16))
        pts_y_black = np.random.randint(0, tf.shape(img)[1]-1, int(tf.shape(img)[0]**2/16))

        img = np.array(img)
        img[pts_x_white, pts_y_white] = [255, 255, 255]
        img[pts_x_black, pts_y_black] = [0, 0, 0]

        return img

    def fit(self, img):
        augs = [self.Cutout_tf, self.WhiteNoise_tf, self.SaltNoise_tf]

        num = np.random.randint(low=1, high=len(augs))
        selected_augs = random.sample(augs, num)
        # for aug in selected_augs:
        #     img = aug(img)
        img = self.Cutout_tf(img)

        return img

    @tf.function
    def Cutout_tf(self, img):
        img = tf.py_function(self.Cutout, [img], [tf.float32])
        return img

    @tf.function
    def WhiteNoise_tf(self, img):
        img = tf.py_function(self.WhiteNoise, [img], [tf.float32])
        return img

    @tf.function
    def SaltNoise_tf(self, img):
        img = tf.py_function(self.SaltNoise, [img], [tf.float32])
        return img

    @tf.function
    def fit_tf(self, img):
        img = tf.py_function(self.fit, [img], [tf.float32])
        return img

In [61]:
class data_aug:
    def resize(self, input_image, real_image, height, width):
        input_image = tf.image.resize(input_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        real_image = tf.image.resize(real_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        return input_image, real_image

    def random_crop(self, input_image, real_image):
        stack_image = tf.stack([input_image, real_image], axis=0)
        cropped_image = tf.image.random_crop(stack_image, size=[2, 256, 256, 3])

        return cropped_image[0], cropped_image[1]

    @tf.function()
    def random_jitter(self, input_image, real_image):
        input_image, real_image = self.resize(input_image, real_image, 286, 286)
        input_image, real_image = self.random_crop(input_image, real_image)

        if tf.random.uniform(()) > 0.5:
            input_image = tf.image.flip_left_right(input_image)
            real_image = tf.image.flip_left_right(real_image)

        return input_image, real_image

In [63]:
# dataset
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

@tf.function
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image

def normalize(input_image, real_image):
    input_image = (input_image / 127.5) - 1
    real_image = (real_image / 127.5) - 1

    return input_image, real_image

def resize(input_image, real_image, height, width):
    input_image = tf.image.resize(input_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image = tf.image.resize(real_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, real_image

def random_crop(input_image, real_image):
    stacked_image = tf.stack([input_image, real_image], axis=0)
    cropped_image = tf.image.random_crop(
        stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image[0], cropped_image[1]

def random_jitter(input_image, real_image):
    # resizing to 286 x 286 x 3
    input_image, real_image = resize(input_image, real_image, 286, 286)

    # randomly cropping to 256 x 256 x 3
    input_image, real_image = random_crop(input_image, real_image)

    if tf.random.uniform(()) > 0.5:
      # random mirroring
        input_image = tf.image.flip_left_right(input_image)
        real_image = tf.image.flip_left_right(real_image)

    return input_image, real_image

def load_image_train(image_file):
    aug = data_aug()
    cr_input = create_input()
    input_image = load(image_file)
    real_image = load(image_file)
    print(input_image)
    input_image = cr_input.fit(input_image)[0]
    print(input_image)
    input_image, real_image = random_jitter(input_image, real_image)
    input_image, real_image = resize(input_image, real_image, 286, 286)
    input_image, real_image = normalize(input_image, real_image)

    return input_image, real_image

def load_image_test(image_file):
    aug = data_aug()
    cr_input = create_input()
    real_image = load(image_file)

    input_image = cr_input.fit(real_image)

    input_image, real_image = aug.resize(input_image, real_image)
    input_image, real_image = normalize(input_image, real_image)

    return input_image, real_image

train_dataset_path = os.listdir('data/train/')
train_dataset_path = [os.path.join(os.path.abspath('data/train/'), p) for p in train_dataset_path]
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset_path)
train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(1)
first_batch = next(iter(train_dataset))
# sample_images = first_batch[0].numpy().reshape((-1, 286, 286))
# sample_image = first_batch[0].numpy().reshape((286, 286, 3))
# cv2.imwrite("/home/naoki/strange.png", sample_image)
# array_obj = plt.imread("/home/naoki/strange.png")
# print(array_obj.shape)
# plt.imshow(array_obj)
# print(sample_image==array_obj)
# train_dataset = tf.data.Dataset.list_files("/home/naoki/Documents/GAN_practice/data/train/*.png")
# print(type(train_dataset))
# train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
# train_dataset = train_dataset.shuffle(BUFFER_SIZE)
# train_dataset = train_dataset.batch(BATCH_SIZE)

# test_dataset = tf.data.Dataset.list_files('data/test/*.png')
# test_dataset = test_dataset.map(load_image_test)
# test_dataset = test_dataset.batch(BATCH_SIZE)

Tensor("PartitionedCall:0", shape=(286, 286, 3), dtype=float32)
Tensor("StatefulPartitionedCall:0", dtype=float32)


ValueError: in converted code:

    <ipython-input-57-18d061b33c91>:56 load_image_train  *
        input_image, real_image = random_jitter(input_image, real_image)
    <ipython-input-43-2a3ecff6315b>:36 random_jitter  *
        input_image, real_image = resize(input_image, real_image, 286, 286)
    <ipython-input-43-2a3ecff6315b>:22 resize  *
        input_image = tf.image.resize(input_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    /home/naoki/anaconda3/envs/tensorflow-new/lib/python3.7/site-packages/tensorflow_core/python/ops/image_ops_impl.py:1357 resize_images_v2
        skip_resize_if_same=False)
    /home/naoki/anaconda3/envs/tensorflow-new/lib/python3.7/site-packages/tensorflow_core/python/ops/image_ops_impl.py:1072 _resize_images_common
        raise ValueError('\'images\' contains no shape.')

    ValueError: 'images' contains no shape.


In [67]:
# dataset
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

@tf.function
def load(image_file):
    image = tf.io.read_file(image_file)
    image = tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [286, 286], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image

def normalize(input_image, real_image):
    input_image = (input_image / 127.5) - 1
    real_image = (real_image / 127.5) - 1

    return input_image, real_image

def resize(input_image, real_image, height, width):
    input_image = tf.image.resize(input_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    real_image = tf.image.resize(real_image, [height, width], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    return input_image, real_image

def random_crop(input_image, real_image):
    stacked_image = tf.stack([input_image, real_image], axis=0)
    cropped_image = tf.image.random_crop(
        stacked_image, size=[2, IMG_HEIGHT, IMG_WIDTH, 3])

    return cropped_image[0], cropped_image[1]

def random_jitter(input_image, real_image):
    # resizing to 286 x 286 x 3
    input_image, real_image = resize(input_image, real_image, 286, 286)

    # randomly cropping to 256 x 256 x 3
    input_image, real_image = random_crop(input_image, real_image)

    if tf.random.uniform(()) > 0.5:
      # random mirroring
        input_image = tf.image.flip_left_right(input_image)
        real_image = tf.image.flip_left_right(real_image)

    return input_image, real_image

def load_image_train(image_file):
    input_image = load(image_file)
    real_image = load(image_file)
    input_image, real_image = random_jitter(input_image, real_image)
    input_image, real_image = normalize(input_image, real_image)

    return input_image, real_image

train_dataset_path = os.listdir('data/train/')
train_dataset_path = [os.path.join(os.path.abspath('data/train/'), p) for p in train_dataset_path]
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset_path)
train_dataset = train_dataset.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [22]:
a = ['abcdefg', 'abcdefghijklmn']
a = tf.data.Dataset.from_tensor_slices(a)

<TensorSliceDataset shapes: (), types: tf.string>


In [32]:
img_path = "/home/naoki/ros/aerial_robot_demo_ws/src/aerial_robot_demo/mbzirc2020/mbzirc2020_task2/mbzirc2020_task2_tasks/data/all/"
files = os.listdir(img_path)
files_file = [f for f in files if os.path.isfile(os.path.join(img_path, f))]  #  all image names
random.shuffle(files_file)
files_file_custom = [img_path+files_file[i] for i in range(len(files_file))]  #  all image path
print(type(files_file_custom))
def load_image(path):
    print(path)
    image = tf.io.read_file(path)
    image = tf.io.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32)
    # image = tf.image.resize(image, (224, 224))
    print(image.shape)
    return image
SPLIT_NUM = 0.7
split_idx = int(0.7*len(files_file))

train_image_path_ds = tf.data.Dataset.from_tensor_slices(files_file_custom[0:split_idx])
train_image_ds = train_image_path_ds.map(load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

print(type(train_image_ds))

<class 'list'>
Tensor("args_0:0", shape=(), dtype=string)
(None, None, 3)
<class 'tensorflow.python.data.ops.dataset_ops.ParallelMapDataset'>


In [4]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0, 0.02)

    result = tf.keras.Sequential()
    result.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding="same", kernel_initializer=initializer, use_bias=False))

    if apply_batchnorm:
        result.add(tf.keras.layers.BatchNormalization())

    result.add(tf.keras.layers.LeakyReLU())

    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0, 0.02)

    result = tf.keras.Sequential()
    result.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding="same", kernel_initializer=initializer, use_bias=False))
    result.add(tf.keras.layers.BatchNormalization())

    if apply_dropout:
        result.add(tf.keras.layers.Dropout(0.5))

    result.add(tf.keras.layers.ReLU())

    return result


In [8]:
a = [i for i in range(5)]
b = [1, 2]
c = a+b
print(c)

[0, 1, 2, 3, 4, 1, 2]


In [48]:
class Generator(tf.keras.Model):
    def __init__(self):
        super(Generator, self).__init__()
        self.down_stack = [
            downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
            downsample(128, 4), # (bs, 64, 64, 128)
            downsample(256, 4), # (bs, 32, 32, 256)
            downsample(512, 4), # (bs, 16, 16, 512)
            downsample(512, 4), # (bs, 8, 8, 512)
            downsample(512, 4), # (bs, 4, 4, 512)
            downsample(512, 4), # (bs, 2, 2, 512)
            downsample(512, 4), # (bs, 1, 1, 512)
        ]

        self.up_stack = [
            upsample(512, 4, apply_dropout=True), # (bs, 2, 2, 1024)
            upsample(512, 4, apply_dropout=True), # (bs, 4, 4, 1024)
            upsample(512, 4, apply_dropout=True), # (bs, 8, 8, 1024)
            upsample(512, 4), # (bs, 16, 16, 1024
            upsample(256, 4), # (bs, 32, 32, 512)
            upsample(128, 4), # (bs, 64, 64, 256)
            upsample(64, 4), # (bs, 128, 128, 128)
        ]

        self.last = tf.keras.layers.Conv2DTranspose(3, 4, strides=2,padding='same', kernel_initializer=tf.random_normal_initializer(0, 0.02), activation='tanh') # (bs, 256, 256, 3)

    def call(self, inputs, training=True):
        x = inputs

        skips = []
        for down in self.down_stack:
            x = down(x)
            skips.append(x)
        skips = reversed(skips[:-1])

        for up, skip in zip(self.up_stack, skips):
            x = up(x)
            x = tf.keras.layers.Concatenate()([x, skip])

        output = self.last(x)

        return output

In [49]:
generator = Generator()
generator.build(input_shape=(None, 256, 256, 3))
generator.summary()

Model: "generator_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_237 (Sequential)  multiple                  3072      
_________________________________________________________________
sequential_238 (Sequential)  multiple                  131584    
_________________________________________________________________
sequential_239 (Sequential)  multiple                  525312    
_________________________________________________________________
sequential_240 (Sequential)  multiple                  2099200   
_________________________________________________________________
sequential_241 (Sequential)  multiple                  4196352   
_________________________________________________________________
sequential_242 (Sequential)  multiple                  4196352   
_________________________________________________________________
sequential_243 (Sequential)  multiple                 

In [39]:
LAMDA = 100
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)

    # mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target-gen_output))
    total_gen_loss = gan_loss + (LAMDA * l1_loss)

    return total_gan_loss, gan_loss, l1_loss

In [50]:
# Discriminator
class discriminator(tf.keras.Model):
    def __init__(self):
        super(discriminator, self).__init__()

        self.down_stack = [downsample(64, 4, False), downsample(128, 4), downsample(256, 4)]
        self.zero_pad1 = tf.keras.layers.ZeroPadding2D()
        self.zero_pad2 = tf.keras.layers.ZeroPadding2D()
        self.conv = tf.keras.layers.Conv2D(512, 4, strides=1, kernel_initializer=tf.random_normal_initializer(0, 0.02), use_bias=False)
        self.batchnorm = tf.keras.layers.BatchNormalization()
        self.leaky_relu = tf.keras.layers.LeakyReLU()
        self.last = tf.keras.layers.Conv2D(1, 4, strides=1, kernel_initializer=tf.random_normal_initializer(0, 0.02))

    def call(self, inputs):
        # inputs = input + target(concatenated)
        x = inputs
        for down in self.down_stack:
            x = down(x)
        x = self.conv(self.zero_pad1(x))
        x = self.zero_pad2(self.leaky_relu(self.batchnorm(x)))
        output = self.last(x)

        return output

In [51]:
disc = discriminator()
disc.build(input_shape=(None, 256, 256, 3))
disc.summary()

Model: "discriminator_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_252 (Sequential)  multiple                  3072      
_________________________________________________________________
sequential_253 (Sequential)  multiple                  131584    
_________________________________________________________________
sequential_254 (Sequential)  multiple                  525312    
_________________________________________________________________
zero_padding2d_8 (ZeroPaddin multiple                  0         
_________________________________________________________________
zero_padding2d_9 (ZeroPaddin multiple                  0         
_________________________________________________________________
conv2d_149 (Conv2D)          multiple                  2097152   
_________________________________________________________________
batch_normalization_238 (Bat multiple              

In [52]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(disc_real_output, disc_generated_output):
    real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)
    generated_loss = loss_object(tf.zeros_like(disc_generated_output), desc_generated_output)
    total_disc_loss = real_loss + generated_loss

    return total_disc_loss

In [53]:
import datetime
log_dir="logs/"

summary_writer = tf.summary.create_file_writer(
    log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [54]:
# training
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

@tf.function
def train_step(input_image, target, epoch):
    with tf.GradientTape() as gen_tape, tf.GradientTape as disc_tape:
        gen_output = generator(input_image, training=True)

        disc_input1 = tf.keras.layers.Concatenate()[input_image, target]
        disc_input1 = tf.keras.layers.Concatenate()[input_image, gen_output]

        disc_real_output = discriminator(disc_input1, training=True)
        disc_generated_output = discriminator(disx_input2, training=True)

        gen_total_loss, gen_gan_loss, gen_l1_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

    generator_gradients = gen_tape.gradient(gen_total_loss, generator.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss, generator.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

    with summary_writer.as_default():
        tf.summary.scalar('gen_total_loss', gen_total_loss, step=epoch)
        tf.summary.scalar('gen_gan_loss', gen_gan_loss, step=epoch)
        tf.summary.scalar('gen_l1_loss', gen_l1_loss, step=epoch)
        tf.summary.scalar('disc_loss', disc_loss, step=epoch)

In [18]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(128, activation='relu')
        self.fc2 = tf.keras.layers.Dense(10, activation='softmax')
        self.dropout = tf.keras.layers.Dropout(0.2)

    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x, training=training)
        x = self.fc2(x)
        return x

model = MyModel()
model.build((None, 28, 28))
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
_________________________________________________________________
dropout_12 (Dropout)         multiple                  0         
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [11]:
a = [1, 2, 3]
b = reversed(a[:-1])

2
1
